<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [26]</a>'.</span>

<img src="../images/AzPTravel_PPM.png">

## GPM Data Transformation Script

#### This script transforms the single consolidated raw file "{Data Collection}.csv" into it's final GPM input versions "{Data Collection Code}localcur.csv" and "{Data Collection Code}euroconv.csv"
#### Current transformations

-    make headers lowercase and replace spaces with hyphens
-    Remove any rows with null BUs
-    validate columns in validcols
-    output documented here: "Global_Attribute_Catalog.xlsx" you can also edit dataframe entries using the instructions in the file
-    replace various null or placeholder values with "Not Provided"


###

In [1]:
commit = ""
# give a reason for the run

percode = "20XX.QX"
# Data Collection Code, this controls file paths and output names

run_type = 1
#run_type =  0 - lite run with no reporting, not recommended.
#run_type =  1 - lite run with normal reporting, default setting.
#run_type =  2 - Heavy run with full reporting, available for audits and troubleshooting.

specialchars = "-GTHtest"
# optional - add up to a 12 character code in order to mark your instance record .ipynb

from datetime import datetime
inst_datetime = datetime.now().strftime("%m%d%Y%H%M%S")
# a single datetime stamp for the full instance run


In [2]:
# Parameters
run_control = 1
percode = "2021.Q1"
commit_message = "Live run, added AT, bad file name, rerun."
inst_datetime = "06012021172659"


### Set Run Control

##### 0 - lite run with no reporting, not recommended.
##### 1 - lite run with normal reporting, default setting.
##### 2 - Heavy run with full reporting, available for audits and troubleshooting.

In [3]:
run_control = 0  # in development mode

In [4]:
#### Packages used
import sys
import os
import pandas as pd
from pandas import ExcelWriter
from numpy import nan

In [5]:
default_dc = "2021.Q1"

try:
    if sys.argv[1] == "-f":
        percode = default_dc
    else:
        percode = sys.argv[1]

except IndexError:
    percode = default_dc
except NameError:
    percode = default_dc

#### style settings

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Convert paths and files to variables.
#### Read data into pd DataFrames.
#### Make paths for the live sources.

In [7]:
rt_path = os.path.join( f'\\\\hecate\\Insurance_US\\Product Development\\Product Management\\Global PPM\\Reporting\\Data Collection\\Production',percode)

infile = os.path.join(rt_path, f'{str(percode)}.parquet')
gppm_file = os.path.join(rt_path, f'GPPM_Input_{str(percode)}.xlsx')
atcat = os.path.join(rt_path, f'{percode}_Attribute_Catalog.xlsx')


In [8]:
#### Read input file

#### This reads the consolidated file that will be transformed or a stored dataframe based on the execution method.

In [9]:
try:
    %store -r read_dc_df
    input = read_dc_df
        
except:
    input = pd.read_parquet(infile,engine = "pyarrow")
    os.remove(infile)

print(input)

                       Submission File Business Unit Country Currency  \
0     us_dat_at_+te_q1_2021_at.parquet            AT      AT      EUR   
1     us_dat_at_+te_q1_2021_at.parquet            AT      AT      EUR   
2     us_dat_at_+te_q1_2021_at.parquet            AT      AT      EUR   
3     us_dat_at_+te_q1_2021_at.parquet            AT      AT      EUR   
4     us_dat_at_+te_q1_2021_at.parquet            AT      AT      EUR   
...                                ...           ...     ...      ...   
3841  us_dat_pt_05202021121056.parquet            PT      PT      EUR   
3842  us_dat_pt_05202021121056.parquet            PT      PT      EUR   
3843  us_dat_pt_05202021121056.parquet            PT      PT      EUR   
3844  us_dat_pt_05202021121056.parquet            PT      PT      EUR   
3845  us_dat_pt_05202021121056.parquet            PT      PT      EUR   

                               Region Reporting Date From Reporting Date To  \
0     North, Central & Eastern Europe       

#### Make a copy for debug purposes

In [10]:
trandata = input.copy()

#### Format Headers

In [11]:
# save the column names to variable, format them, replace headers
cols = trandata.columns.values

"""format headers"""
fixedcols = []

for f in cols:
    fixedhead = f.replace(' ', '_').lower()
    fixedcols.append(fixedhead)

trandata.columns = fixedcols

'format headers'

#### Update Verification 1

In [12]:
if run_control > -10:
    disp = {"Original": input.columns, "Transformed": trandata.columns}

    disp = pd.DataFrame(disp)

    print(disp.to_markdown())
else:
    print("Skipped Transformation Report 1")

|    | Original                                                        | Transformed                                                     |
|---:|:----------------------------------------------------------------|:----------------------------------------------------------------|
|  0 | Submission File                                                 | submission_file                                                 |
|  1 | Business Unit                                                   | business_unit                                                   |
|  2 | Country                                                         | country                                                         |
|  3 | Currency                                                        | currency                                                        |
|  4 | Region                                                          | region                                                          |
|  5 | Reporting Date From 

#### Remove rows with null business units


In [13]:
# todo, remove this from read file
trandata = trandata[trandata.business_unit.notnull()]

#### Update Verification 2

In [14]:
if run_control > -10:

    ibus = input.groupby(['Business Unit']).count()
    ibus = ibus.reset_index()
    tbus = trandata.groupby(['business_unit']).count()
    tbus = tbus.reset_index()

    compdata = pd.merge(ibus, tbus, left_on='Business Unit', right_on='business_unit', how='left')

    disp = {"Business Unit": compdata['Business Unit'], "Original": compdata['Earned Revenues net of Taxes'],
            "Transformed": compdata['earned_revenues_net_of_taxes']}

    disp = pd.DataFrame(disp)

    print("Row Counts")
    print(disp.to_markdown())
    
else:
    
    print("Skipped Transformation Report 2")


Row Counts
|    | Business Unit   |   Original |   Transformed |
|---:|:----------------|-----------:|--------------:|
|  0 | AT              |        222 |           222 |
|  1 | AU              |         53 |            53 |
|  2 | CA              |         17 |            17 |
|  3 | CH              |       3187 |          3187 |
|  4 | CZ              |          7 |             7 |
|  5 | ES              |         28 |            28 |
|  6 | GR              |         15 |            15 |
|  7 | IT              |        100 |           100 |
|  8 | NL              |        151 |           151 |
|  9 | PL              |         27 |            27 |
| 10 | PT              |         39 |            39 |


#### Swap  nulls for "Not Provided" in 'Sub Lob' and 'Distribution Channel'

In [15]:
mults = trandata['sub_lob'][trandata.sub_lob.replace(nan, 'Not Provided').str.contains(',', case=False)]
mults = mults.unique()

trandata['sub_lob'] = trandata['sub_lob'].fillna('Not Provided')
trandata['sub_lob'] = trandata['sub_lob'].replace('', 'Not Provided')
trandata['distribution_channel'] = trandata['distribution_channel'].replace('', 'Not Provided').fillna('Not Provided')

#### Replace multiple entries in "Sub Lob" to "Multiple"

In [16]:
for i in mults:
    trandata['sub_lob'] = trandata['sub_lob'].replace(i, 'Multiple')

# todo automate figuring out which fields fx rates should be applied to somehow

#### Update Verification 3

In [17]:
if run_control > -10:

    ibus = input.groupby(['Sub LOB']).sum()
    ibus = ibus.reset_index()
    tbus = trandata.groupby(['sub_lob']).sum()
    tbus = tbus.reset_index()

    compdatasublob = pd.merge(ibus, tbus, left_on='Sub LOB', right_on='sub_lob', how='outer')

    disp1 = pd.DataFrame(
        {'Original Sub LOB': compdatasublob['Sub LOB'], 'Transformed Sub LOB': compdatasublob['sub_lob'],
         "Original": compdatasublob['Earned Revenues net of Taxes'] / 1000,
         "Transformed": compdatasublob['earned_revenues_net_of_taxes'] / 1000})



    tots1 = pd.DataFrame({'Original Total': (input['Earned Revenues net of Taxes']).sum() / 10000,
                          "Transformed Total": (compdatasublob['earned_revenues_net_of_taxes']).sum() / 10000},
                         index=[0])
else:
    print("Skipped Transformation Report 2")

#### Make cflds, a list of currency fields, force to float, coercion is null or string to 0

In [18]:
cflds = ['written_revenues_net_of_taxes', 'written_revenues', 'earned_revenues_net_of_taxes',
         'earned_revenues', 'earned_base_commissions', 'earned_over-commissions', 'upfront_cash_payments',
         'total_compensation', 'paid_claims', 'ocr_+_ibnr', 'actual_incurred_losses_(paid_+_ocr_+_ibnr)',
         'internal_variable_costs_(excl._az_tech_fee)', 'az_tech_fee', 'internal_fixed_costs_(excl._hq_fees)',
         'hq_fees', 'total_expenses', 'risk_premium', 'profit_or_loss', 'contribution_margin_-_hq_view',
         'contribution_margin_-_bu_view', ]

for i in cflds:
    pd.to_numeric(trandata[i], errors='coerce')

0           0.00
1           0.00
2           0.00
3       31328.10
4         887.19
          ...   
3841     1509.16
3842     2164.11
3843      309.72
3844        0.00
3845     1260.00
Name: written_revenues_net_of_taxes, Length: 3846, dtype: float64

0           0.0000
1           0.0000
2           0.0000
3       34776.0000
4         998.0800
           ...    
3841     1644.9844
3842     2358.8799
3843      337.5948
3844        0.0000
3845     1373.4000
Name: written_revenues, Length: 3846, dtype: float64

0           0.00
1           0.00
2           0.00
3       31328.10
4         887.19
          ...   
3841     1576.86
3842     2085.80
3843       16.57
3844       68.00
3845     1260.00
Name: earned_revenues_net_of_taxes, Length: 3846, dtype: float64

0           0.0000
1           0.0000
2           0.0000
3       34776.0000
4         998.0800
           ...    
3841     1718.7774
3842     2273.5220
3843       18.0613
3844       74.1200
3845     1373.4000
Name: earned_revenues, Length: 3846, dtype: float64

0          0.00
1          0.00
2          0.00
3       4391.77
4        406.87
         ...   
3841       0.00
3842       0.00
3843       0.00
3844       0.00
3845       0.00
Name: earned_base_commissions, Length: 3846, dtype: float64

...

0          0.000000
1          0.000000
2          0.000000
3       8959.836600
4        253.736340
           ...     
3841     848.977244
3842     890.287119
3843     309.417018
3844      58.430911
3845    1289.873953
Name: total_expenses, Length: 3846, dtype: float64

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3841    0.0
3842    0.0
3843    0.0
3844    0.0
3845    0.0
Name: risk_premium, Length: 3846, dtype: float64

0       49247.040000
1         100.000000
2        1100.000000
3       16035.553400
4         910.583660
            ...     
3841     -813.677244
3842      586.792881
3843     -922.967018
3844        9.569089
3845      -29.873953
Name: profit_or_loss, Length: 3846, dtype: float64

0       49247.040000
1         100.000000
2        1100.000000
3       21975.925066
4        1078.810853
            ...     
3841     -443.903574
3842     1075.912981
3843     -919.081353
3844       32.927089
3845      388.446047
Name: contribution_margin_-_hq_view, Length: 3846, dtype: float64

0       49247.040000
1         100.000000
2        1100.000000
3       20629.108566
4        1054.794853
            ...     
3841     -540.880464
3842      947.636281
3843     -920.100408
3844       28.745089
3845      325.446047
Name: contribution_margin_-_bu_view, Length: 3846, dtype: float64

#### Turn selected columns values uppercase

In [19]:
validcols = ['business_unit', 'currency', 'region', 'type_of_analysis', 'type_of_business', 'type_of_account', 'lob',
             'distribution_type', 'distribution_channel', ]

for c in validcols:
    trandata[c] = trandata[c].astype(str)
    trandata[c] = trandata[c].apply(lambda x: x.upper())

#### Read the values from the Global Attribute Catalog, one field per loop iterance and xlsx sheet.

In [20]:
checktabs = []



for s in validcols:
    t = pd.read_excel(atcat, sheet_name = s )
    checktabs.append([[s], [t]])

gacout = []


#### Compare the lists from the previous step and each field. Find those that do not match, make lists of unique values
#### replace any that have replacements in GPPM inoout already, make a list of values without any matches.


In [21]:
# todo rewrite this beginner code to use dicts and simplify

for u in enumerate(validcols):
    trandata.loc[0:len(trandata[u[1]]), u[1]] = trandata[u[1]].replace(
        [checktabs[u[0]][1][0]['Upper_Vers']][0].to_numpy(), [checktabs[u[0]][1][0][u[1]]][0].to_numpy())
    d = list(checktabs[u[0]][1][0]['Non Matches'].drop_duplicates().dropna().append(
        pd.Series(trandata[u[1]][~trandata[u[1]].isin(checktabs[u[0]][1][0][u[1]])].drop_duplicates().dropna()),
        ignore_index=True))
    g = [checktabs[u[0]][1][0][str(u[1])], checktabs[u[0]][1][0]['Upper_Vers'], checktabs[u[0]][1][0]['Unnamed: 2'],
         pd.Series(d, dtype='object').drop_duplicates().dropna(), checktabs[u[0]][1][0]['User Defined Corrections']]
    gacout.append(list([g, u[1]]))

#### Rebuild the GPPM input file, with the replaced values in the bad values column of each sheet.
#### Step one, make and xlsx with a notes page.

In [22]:
w = ExcelWriter(atcat)
notes = pd.DataFrame([
    "This page is script generated during the source creation process. Do not edit these notes directly in the file as they will be overwritten",
    "", ""
    , " Purpose :     to manage attribute entries in the data collection process, this workbook documents and organizes all entries and also allows a user to swap those that do \
                        not conform to validation rules  with an entry of their choice",
    " Each attribute field that requires validation has its own sheet tab", ""
    , "Column A:    of each sheet tab contains all unique acceptable responses",
    "Column B:    an upper case version, to wrangle case mismatches",
    "Column D:   is generated by the process, this is a list of an uppercase version of each unique unacceptable response, this builds over time with each collection",
    "Column E:   you can enter accpetable response here (sase sensitive) to be swapped out in the data, save and exit this file, run the process again and they will be replaced"
    , "", "", "Gavin Harmon 9 - July -2020"])

notes.columns = ['Notes']

notes.to_excel(w, index=False, sheet_name="Notes")

#### Step two, build the new sheets for each validcols field

In [23]:
for v in enumerate(validcols):
    df = pd.DataFrame(gacout[v[0]][0], index=[f"{v[1]}", 'Upper_Vers', '', 'Non Matches', 'User Defined Corrections']).T
    df.to_excel(w, index=False, sheet_name=gacout[v[0]][1])
w.save()

#### Step three, read these lists back in, make the necessary replacements in the DataFrame

In [24]:
for s in validcols:
    t = pd.read_excel(f'//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/{str(percode)}\\{str(percode)}_Attribute_Catalog.xlsx', sheet_name = s )
    checktabs.append([[s],[t]])

In [25]:
gacout = []

for u in enumerate(validcols):
    trandata.loc[0:len(trandata[u[1]]), u[1]] = trandata[u[1]].replace(
        [checktabs[u[0]][1][0]['Non Matches']][0].to_numpy(),
        [checktabs[u[0]][1][0]['User Defined Corrections']][0].to_numpy())
repper = trandata[['business_unit', 'reporting_date_to']]
repper = pd.DataFrame({"business_unit": (repper['business_unit']), "YearMo": (repper['reporting_date_to'])})
minrep = repper.groupby(['business_unit']).max()
a = trandata['business_unit'].replace(list(minrep.axes[0]), minrep.get("YearMo"))
trandata['rep_date'] = a
trandata.loc[0:len(trandata['business_partner_id_number']), 'business_partner_id_number'] = trandata[
    'business_partner_id_number'].replace('0', 'Not Provided').fillna('Not Provided')
trandata.loc[0:len(trandata['product_id_number']), 'product_id_number'] = trandata['product_id_number'].replace('0',
                                                                                                                'Not Provided').replace(
    '-', 'Not Provided').fillna('Not Provided')
trandata.loc[0:len(trandata['sub_lob']), 'sub_lob'] = trandata['sub_lob'].replace('0', 'Not Provided').replace('-',
                                                                                                               'Not Provided').replace(
    'Other', 'Not Provided').fillna('Not Provided')

#### Evaluate the current state, alert the user if input is needed.
- At the end of this process, alert the user with instructions if they need to adjust bad entries for validated fields.



<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [26]:
#see above note about converting to dictionaries

#Make lists of all the attribute catalog entries that do not have a correction as a list of errors.
err_messages = []

for idx, i in enumerate(checktabs):
    subset_no_nans =  i[1][0][['Non Matches', 'User Defined Corrections']][~i[1][0]['Non Matches'].isna()]
    entries = subset_no_nans['Non Matches'][subset_no_nans['User Defined Corrections'].isna()]. tolist()
    if len(entries) > 0: 
        err_messages.append(f'Please deal with bad entries {entries} on tab {i[0][0]} of the Attribute Catalog.')

#raise an error to stop the process and give instructions

# define Python user-defined exceptions


# define Python user-defined exceptions
class Error(Exception):
    """Base class for other exceptions"""
    pass

class BadEntriesError(Error):
    """Raised when the input value is too large"""
    for i in err_messages : print(i)
    
    os.startfile(atcat)

if len(err_messages) > 0:
    raise BadEntriesError
else:
    pass
        

Please deal with bad entries ['0.0'] on tab distribution_type of the Attribute Catalog.
Please deal with bad entries ['0.0'] on tab distribution_channel of the Attribute Catalog.


BadEntriesError: 

#### Remove empty rows, if there is no claims experience and no revenue for a 12 month period, it should not be included

In [ ]:
trandata = trandata.loc[(trandata['units_of_risk_(written)'].fillna(0).replace('', 0)
                         + trandata['written_revenues_net_of_taxes'].fillna(0).replace('', 0)
                         + trandata['written_revenues'].fillna(0).replace('', 0)
                         + trandata['number_of_policies_(earned)'].fillna(0).replace('', 0)
                         + trandata['units_of_risk_(earned)'].fillna(0).replace('', 0)
                         + trandata['earned_revenues_net_of_taxes'].fillna(0).replace('', 0)
                         + trandata['earned_revenues'].fillna(0).replace('', 0)
                         + trandata['earned_base_commissions'].fillna(0).replace('', 0)
                         + trandata['upfront_cash_payments'].fillna(0).replace('', 0)
                         + trandata['earned_over-commissions'].fillna(0).replace('', 0)
                         + trandata['total_compensation'].fillna(0).replace('', 0)
                         + trandata['number_of_claims_(paid_+_ocr_+_ibnr)'].fillna(0).replace('', 0)
                         + trandata['number_of_open_claims'].fillna(0).replace('', 0)
                         + trandata['open_claims_%'].fillna(0).replace('', 0)
                         + trandata['number_of_persons_involved_in_claims_(paid_+_ocr_+_ibnr)'].fillna(0).replace('', 0)
                         + trandata['paid_claims'].fillna(0).replace('', 0)
                         + trandata['ocr_+_ibnr'].fillna(0).replace('', 0)
                         + trandata['actual_incurred_losses_(paid_+_ocr_+_ibnr)'].fillna(0).replace('', 0)
                         + trandata['internal_variable_costs_(excl._az_tech_fee)'].fillna(0).replace('', 0)
                         + trandata['az_tech_fee'].fillna(0).replace('', 0)
                         + trandata['internal_fixed_costs_(excl._hq_fees)'].fillna(0).replace('', 0)
                         + trandata['hq_fees'].fillna(0).replace('', 0)
                         + trandata['total_expenses'].fillna(0).replace('', 0)
                         + trandata['frequency_(earned)'].fillna(0).replace('', 0)
                         + trandata['severity'].fillna(0).replace('', 0))
                        != 0]

#### Output temp file

In [ ]:
# trandata.to_parquet(
#     f'//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/{str(percode)}\\{str(percode)}.localcur.parquet', engine = "pyarrow")

trandata.to_csv(
    
     f'//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/{str(percode)}\\{str(percode)}.localcur.csv', index = False)


#### Store the DataFrame for other noteboks to use

In [ ]:
py_t_df = trandata

%store py_t_df
